# Optimisation des paramètres de simulation avec Hyperopt

**Contexte:**
    
Dans ce notebook, nous cherchons à produire des Time-Series de simulation qui se rapproche un maximum des Time-Series de réference.
Pour ce faire, nous utilisons la méthode d'optimisation Hyperopt pour trouver la combinaison de paramètres pour le modèle simplifié qui se rapproche du modèle de réference.

Hyperopt prend l'espace de recherche que l'on lui fournit, utilisé un algorithme de suggestion pour décider des paramètres qu'il souhaite évaluer et enfin Hyperopt utilise une fonction objectif et cherche à minimiser cette fonction objectif.

Les fonctions objectifs utilisable avec Hyperopt sont les fonctions objectifs qui sont minimisables.

## Modules import

First of all we import the dependencies necessary to make the different implementations work.

In [ ]:

import sys
sys.path.append("..")
import yaml
import numpy as np
import pandas as pd

from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error
from hyperopt import hp, fmin, tpe

from uvsw_part.simulation import run_cable_wakeosc

import matplotlib.pyplot as plt
import math

## Définition du chemin vers le fichier Liste*.txt et de le numéro de la Time-Series de réference utilisé

The path to the List * file is used to retrieve the parameters of the reference Time-Series. <br> The index allows you to choose which Time-series we are going to work with (The index starts at 0)

In [ ]:
# Path to list of Time series
LIST_PATH = "../data/params/List1.txt"
# Index of time series to simulate
TS_INDEX = 5

## Chargement de la Time-Series de reférence

Ici, nous chargeons le fichier Liste utilisé ainsi que la Time-Series de réference auquel nous allons comparé la simulation

In [ ]:
# Load list number
LIST_REF = LIST_PATH[-9:-4].lower()

# Read table of time series
data_list = pd.read_csv(LIST_PATH , delim_whitespace=True)
# Get parameters of selected time series
set_params = data_list.iloc[TS_INDEX,:]
# Get dataframe of data of corresponding TS

#Reference time-series loading
ref = pd.read_csv("../data/ref/"+ LIST_REF + "/graph{}.csv".format(set_params["nc"]))

In [ ]:
# Show params of selected TS
set_params

## Df and Dt parameters setup to obtain Time-Series from the simulator with the same number of points as the reference model

By setting the dt and dr value to the value of tf (Final Time) divided by the number of points in the reference model, we can obtain a simulation whose has the same number of points as the reference model. <br> This allows the use of methods to compare the 2 (RMSE,MSE, R2_SCORE).

In [ ]:
# Definition of parameters allowing to obtain simulation Time-Series of the same length as the reference Time-Series
tf_val = set_params["tf[s]"]
dt_val = tf_val / len(ref)
dr_val = tf_val / len(ref)

## Hyperopt search space definition

Here we define the search space given to Hyperopt by defining the parameters and the distribution of values. <br> By After we define this search space variable, we give it to Hyperopt to try to find the set of parameters that minimizes the objective function.
Set the pp parameter to False to avoid the mass prints of every iteration

In [ ]:
search_space = {
    'cable': {
        'type': 'None',
        'length': 1.0,
        'tension': hp.uniform('tension_val',100.0,40000.0),
        'h': hp.uniform('h_val',0.0,1000.0)},
    'conductor': {'m': 1.57, 'd': 0.025, 'EA': 0.0},
    'simulation': {'ns': 101,
        'tf': tf_val,
        'dt': dt_val,
        'dr': dr_val,
        'si': 99,
        'pp': False}, ####### Enlever les prints
    'wakeosc': {'u': hp.uniform('u_val',0.05,7.00),
        'st': 0.235,
        'cl0': 0.6,
        'eps': 0.3,
        'al': 0.1,
        'bt': 0.0,
        'gm': 0.0,
        'md': 1,
        'y0': 0.0,
        'q0': 0.05}
}

## Data coverage rate

We calculate a metric allowing to know the rate of values resulting from simulation included within the reference time-series for each timesteps. <br> We compare the sign of the value from the simulation and the reference for each values, and if they both share the same sign, we compare whether the simulation value has a lower amplitude than the reference one.

In [ ]:
def t_couverture (ref,dfy):
    """
    Check if the reference and simulation values have the same sign one by one
     If same sign, appropriate operations to see if the amplitude of the negative or positive simulation
     is well contained in the reference 
    
    Arguments :
    ref (pandas.core.series.Series) (numpy.ndarray) : Reference model 
    dfy (numpy.ndarray)                             : Simplified model generated by the simulator 
    
    
    Sortie :
        Data coverage rate (float): Pourcentage de valeurs de la simulation 
        à l'intérieur de la réference   
    """
    same_sign = 0 # Number of the same sign in reference and simulation (taken 1 by 1)
    
    # Number of times the simulation has the same sign as the reference 
    # AND with the simulation contained in the reference (lower ampltude) 
    
    couverture = 0
    
    
    ref_taille = len(ref) # Total number of values in the reference or the simulation
    
    #Debug
    diff_sign = 0 # Number of times the sign differs between reference and simulation (taken 1 by 1) 
    
    
    for i in range(0,ref_taille):
        if(np.sign(ref[i]) == np.sign(dfy[i])): #If the reference and the sim have the same sign
            same_sign = same_sign + 1


            if(ref[i] < 0): # When the reference < 0  
                if(ref[i] - dfy[i] < 0):# Check if the reference has bigger negative amplitude than the simulation 
                    couverture = couverture + 1
                

            if(ref[i] > 0): # When the reference > 0 
                if(ref[i] - dfy[i] > 0):# Check if the reference has bigger positive amplitude than the simulation 
                    couverture = couverture + 1

            if(ref[i] == 0): # Quand la reference = 0
                if(ref[i] - dfy[i] == 0): # Check if the reference has same amplitude as the simulation
                    couverture = couverture + 1

            if(math.isnan(ref[i]) and math.isnan(dfy[i]) == True): # When the values of the reference and the simulation are both are not available (Na) 
                couverture = couverture + 1             
            
        else :
            diff_sign = diff_sign + 1 # If the reference and the simulation do not have the same sign (debug) 
            

    taux_couverture = (couverture / ref_taille) * 100 # Data coverage rate
    return taux_couverture

## Definition of objective functions used

Here, we define the objective functions which are used with Hyperopt in order to retrieve combinations of simulator parameters approaching the reference model. <br>
The objective functions calculate metrics which are used to assess the similarity between the simulation Time-Series and the reference one. <br>
The metrics obtained must be minimized by Hyperopt.
The 2 metrics used are the RMSE (Root Mean Squared Error) and the 1 - R2_score

## Objective Function 1: Root Mean Squared Error (RMSE)

The goal function used here uses a metric based on the RMSE. We calculate the RMSE between the simulation and the reference to compare the results. <br>The closer the result is to 0, the more similar the results. <br> This metric is less performant than the one based on the 1 - R2_SCORE
<p>
The simulation creation must be deleted after usage before the end of the function, to avoid exponential memory consumption and crash

In [ ]:
def rmse_sim(cfg):
    """

    Function allowing to calculate an objective function based on the RMSE and to display the graphs 
    for each iteration of Hyperopt
    
    Arguments : Search space 
    
    Sortie : Score (float): calculated with the cost function (RMSE)
              Plot with the values proposed by Hyperopt 
    
    """
    # Run simulation
    dfy, _ = run_cable_wakeosc(cfg)
    print("yes")
    # Current config
    print("#"*100)
    print("Current parameters:")
    print("\tU value: {}, \n\tH value: {}, \n\tTension value: {}, \n\tcl0 value: {}, \n\teps value: {}".format(
        cfg['wakeosc']['u'],
        cfg['cable']['h'],
        cfg['cable']['tension'],
        cfg['wakeosc']['cl0'],
        cfg['wakeosc']['eps'])
    )
    print("#"*100)
    
    # Calculate score
    score = np.sqrt(mean_squared_error(ref['y/d'],(dfy['s=0.250']/0.025).values[:-1]))
    
    # Score saved in string for the plot
    score_text = "RMSE = %s " % score

    # Plot graphics
    plt.figure(figsize = (20,5))
    plt.plot(ref['time'], ref['y/d'], label = "Reference signal")
    plt.plot(dfy.index, dfy['s=0.250']/0.025, label = "Simulator signal")
    plt.xlabel('time (s)')
    plt.ylabel('y/d')
    
    # Plot score and parameters
    plt.figtext(0.5, -0.05, score_text, ha="center", fontsize=18, bbox={"facecolor":"orange", "alpha":0.5, "pad":5})
    plt.figtext(0.5, -0.20, "h = '{0}', tension = '{1}', u = '{2}', clo = '{3}', eps = '{4}', st = '{5}' ".format(cfg['cable']['h'],
                                                                                                                  cfg['cable']['tension'],cfg['wakeosc']['u'],cfg["wakeosc"]["cl0"],
                                                                                                                  cfg["wakeosc"]["eps"],cfg["wakeosc"]["st"]), ha="center", fontsize=18, bbox={"facecolor":"orange", "alpha":0.5, "pad":5})
    title = "Comparison of the simulation signal with the parameter values supplied by HYPEROPT with the reference signal, Objective function used: RMSE "
    plt.title(title,fontsize=18)


    plt.legend()
    plt.show() 

    
    print("#"*40)
    print("Current scores:")
    print("\tR2 : {}".format(r2_score(ref['y/d'],(dfy['s=0.250']/0.025).values[:-1])))
    print("\tMSE : {}".format(mean_squared_error(ref['y/d'],(dfy['s=0.250']/0.025).values[:-1])))
    print("#"*40)
    # Delete the simulation
    del dfy
    # Return the score
    return score

## Objective function: based on R2 SCORE 

The objective function defined here is a function based on the R2_SCORE. The R2_SCORE is used to compare the simulation and the reference. <br>
To evaluate the R2_SCORE, the R2_SCORE must tend towards 1 to affirm that the 2 results are close, an R2_SCORE = 0 being bad, and an R2_SCORE < 0 means that a straight line looks more like the reference than the simulation (worse). <p>
However, in order to have a formula that can be minimized by Hyperopt, the computational formula established here is 1.0 - R2_SCORE. This causes the result to approach 0 as it improves.
    
<p>
The simulation creation must be deleted after usage before the end of the function, to avoid exponential memory consumption and crash

In [ ]:
def r2_sim(cfg):
    
    """

    Function allowing to calculate the cost function and to display the graphs for each iteration of hyperopt
    Arguments : search_space
    
    Sortie :Score calculated with the cost function (1 - R2_score) 
            Plot with the values proposed by Hyperopt and metrics
    
    """
    # Run simulation
    dfy, _ = run_cable_wakeosc(cfg)
    print("yes")
    # Current config
    print("#"*100)
    print("Current parameters:")
    print("\tU value: {}, \n\tH value: {}, \n\tTension value: {}, \n\tcl0 value: {}, \n\teps value: {}".format(
        cfg['wakeosc']['u'],
        cfg['cable']['h'],
        cfg['cable']['tension'],
        cfg['wakeosc']['cl0'],
        cfg['wakeosc']['eps'])
    )
    print("#"*100)
    
    # Calculate score
    score = 1.0 - r2_score(ref['y/d'],(dfy['s=0.250']/0.025).values[:-1])
    taux_couverture = t_couverture(ref['y/d'],(dfy['s=0.250']/0.025).values[:-1])
    r2 = r2_score(ref['y/d'],(dfy['s=0.250']/0.025).values[:-1])
    
    # Store score in a string for visualisation purpose
    score_text = "Score (1 - R2) = %s " % score
    r2_text = "R2 = %s " % r2
    taux_couverture_text = "Data coverage rate = %s %%" % taux_couverture
    
    plt.figure(figsize = (20,5))
    plt.plot(ref['time'], ref['y/d'], label = "Signal de reference")
    plt.plot(dfy.index, dfy['s=0.250']/0.025, label = "Signal du simulateur")
    plt.xlabel('time (s)')
    plt.ylabel('y/d')
    
    # Plot score_value and parameters value
    plt.figtext(0.5, -0.05, score_text, ha="center", fontsize=18, bbox={"facecolor":"orange", "alpha":0.5, "pad":5})
    plt.figtext(0.5, -0.20, r2_text, ha="center", fontsize=18, bbox={"facecolor":"orange", "alpha":0.5, "pad":5})

    plt.figtext(0.5, -0.30, "h = '{0}', tension = '{1}', u = '{2}', clo = '{3}', eps = '{4}', st = '{5}' ".format(cfg['cable']['h'],
                                                                                                        cfg['cable']['tension'],cfg['wakeosc']['u'],cfg["wakeosc"]["cl0"],
                                                                                                                  cfg["wakeosc"]["eps"],cfg["wakeosc"]["st"]), ha="center", fontsize=18, bbox={"facecolor":"orange", "alpha":0.5, "pad":5})
    plt.figtext(0.5, -0.40, taux_couverture_text, ha="center", fontsize=18, bbox={"facecolor":"orange", "alpha":0.5, "pad":5})

    title = "Comparison of the simulation signal generated with the parameter values provided by HYPEROPT with the reference signal, Objective function used: R2_Score"
    plt.title(title,fontsize=18)


    plt.legend()
    plt.show() 
    # Compute the score

    
    print("#"*40)
    print("Current scores:")
    print("\tR2 : {}".format(r2_score(ref['y/d'],(dfy['s=0.250']/0.025).values[:-1])))
    print("\tMSE : {}".format(mean_squared_error(ref['y/d'],(dfy['s=0.250']/0.025).values[:-1])))
    print("#"*40)
    # Delete the simulation
    del dfy
    # Return the score
    return score

## Launch of the optimization process with Hyperopt

We launch the Hyperopt algorithm with the chosen objective function, the desired search space, the suggestion algorithm used by Hyperopt and the maximum number of iterations <p>

The mostly used suggestion algorithm is TPE (Tree Parzen Estimator). Our empirical data tells us that TPE execution speed is faster than Simulated Annealing in this case. <p>
    
Our empirical data also suggest that the results become interesting to exploit for iterations >= 500

In [ ]:
# Run the optimization process
best = fmin(r2_sim, search_space, algo=tpe.suggest, max_evals=500)